# Before Start: 


## 字幕下載


1. F12 - network panel - search ?o=

2. look for file start with ?o=

3. right click - open in new tab - would download automatically 

4. change file name as anything.txt  or anything.srt


## 字母處理

1. Convert to srt https://gotranscript.com/subtitle-converter
2. Merge 2 srt file (bilingual) https://subtitletools.com/merge-subtitles-online 
3. Merge setting: Threshold in milliseconds set tobe 3000 mili second

## Before execute code

1. Run under Window OS
2. Merged srt files path should be like `\\text\\merged_subs\\ko_zh\\ko1_zh1.txt`
2. Make all the csv file/path txt file/path before hand, which might be fixxed in some time.

# After execution

## encoding issue
1. csv opened by excel would be broken since encoding issue
2. open new excel - 資料 - CSV（匯入） - select UTF-8
## seperate different language in excel
1. ctrl + H :
  - &lrm   to be 
  - \>
2. 資料 - 資料剖析 - 分隔符號 - 下一步 - 其他欄位内填入">" 
3. ctrl + H : replace "\n", "]" ... etc. Not processed by code since every ttml might have different patterns

In [72]:
import os, re, shutil, csv
import pprint

# /...........helper function............../

          
# /.....merged files looks like this..... /
'''
    01:12:53,952 --> 01:12:55,204
    <c.korean><c.bg_transparent>언제 들어왔어, 삼촌?</c.bg_transparent></c.korean>
    你何時回來的？叔叔
'''


# merged .srt content start with line 1. no captions lines to clean
startLine = 0 

# clean captions lines
def pop_forMerged(fileCopy):
    
    with open(fileCopy, "r+", encoding="UTF-8") as f: # open file in read / write mode
        for i in range(startLine):
            lineCurser = f.readline()    # read the i'th line and throw it out
        text = f.read()                  # store the rest
        cleanText = cleanTag_forMerged(text)
        f.seek(0)                        # set the cursor to the top of the file
        f.write(cleanText)               # write the data back
        f.truncate()                     # set the file size to the current size
        
        
# parsing srt file, leave only sequence number, time stmap, and pain text
# FOR MERGED: clean the Korean context 
def cleanTag_forMerged (text):
    #cleanText = re.sub(r'<c.korean>.*?</c.korean>\n', '', text)
    cleanText = re.sub(r'<.*?>', '', text)
    # &lrm 是故意留的， 爲了讓之後 excel 内 雙語好分開 
    #cleanText = re.sub(r'&lrm;', '', cleanText)
    # \n 都取代掉，toDict() would return empty {}
    #cleanText = re.sub(r'\n', '', cleanText)
    return cleanText





# transfer to dict
def toDict(fileCopy):
    with open(fileCopy, "r", encoding="UTF-8") as f:
        lines = f.readlines()
        print("Dict accessed....")
    
    re_pattern = r'[0-9]{2}:[0-9]{2}:[0-9]{2}.[0-9]{3} -->'   # /.....  r'/d/d:/d/d:/d/d./d/d/d -->' 不能用？ ..../
    regex = re.compile(re_pattern)

    # Get start times in file
    start_times = list(filter(regex.search, lines))
    start_times = [time.split(' ')[0] for time in start_times]
    
    # Get lines in file
    subtitles = [[]]
    for sentence in lines:
        if re.match(re_pattern, sentence):
            subtitles[-1].pop()
            subtitles.append([])
        else:
            subtitles[-1].append(sentence)
    subtitles = subtitles[1:]         

    # Merge results
    linesDict = {start_time:line for start_time,line in zip(start_times, subtitles)}
    # pprint.pprint(linesDict)
    return linesDict


# dict transfer to csv file
def toCsv(csvPath, linesDict, language):
    
    # get Table for sentence
    with open(sentenceCsv, 'w', newline='', encoding='utf-8') as outputFile:
        outputWriter = csv.writer(outputFile)
        outputWriter.writerow(['chunckID', 'start_time', f'subtitles_{language}'])
        chunckID = 0
        for start_time, line in linesDict.items():
            outputWriter.writerow([chunckID, start_time, line])
            chunckID += 1




# /..............main function............./

target_lan = 'zh' # 你現在在執行的是 Korean 跟哪個語言 merged 的檔案?
epNum = 2        # merged 你要16集轉成 csv

for i in range(1, epNum):
    
    # file name & get a copy
    file = os.path.abspath('.') + f'\\text\\merged_subs\\ko_{target_lan}\\' + f'ko{i}_{target_lan}{i}.txt'
    fileCopy = os.path.abspath('.') + '\\text\\merged_subs\\backup\\' + f'ko{i}_{target_lan}{i}_copy.txt'
    shutil.copy(file, fileCopy)
    
    # access copied files
    pop_forMerged(fileCopy)
    linesDict = toDict(fileCopy)
    
    # csv for Sentence Table
    sentenceCsv = os.path.abspath('.') + '\\text\\merged_subs\\csv\\' + f'ko{i}_{target_lan}{i}.csv'
    toCsv(sentenceCsv, linesDict, target_lan)

    print(f"episode:{i} csv for merged subtitles -- Ko & {target_lan} is done!")


# csv to db


Dict accessed....
episode:1 csv for merged subtitles -- Ko & zh is done!


In [74]:
# for debugging:


# pprint.pprint(cleanText)
# pprint.pprint(linesDict)

# ref: Data Comparation

## Raw TTML Data looks like

```xml
NOTE SegmentIndex
NOTE Segment=597.931 40233@720 188
NOTE .....
NOTE /SegmentIndex
                                                                                                                                                                                                                                                                                                 
1
00:00:00.375 --> 00:00:02.919 position:50.00%,middle align:middle size:80.00% line:84.67% 
<c.korean><c.bg_transparent>[주제곡]</c.bg_transparent></c.korean>

2
00:00:52.093 --> 00:00:54.220 position:50.00%,middle align:middle size:80.00% line:10.00% 
<c.korean><c.bg_transparent>(김신)</c.bg_transparent></c.korean>
<c.korean><c.bg_transparent>그래, 그래서 하는 말인데</c.bg_transparent></c.korean>

3
00:00:55.889 --> 00:00:58.641 position:50.00%,middle align:middle size:80.00% line:10.00% 
<c.korean><c.bg_transparent>오늘 날이 좀 적당해서 하는 말인데</c.bg_transparent></c.korean>
```

## Srt converted Data looks like

```xml
11
00:02:19,889 --> 00:02:21,641
<c.korean><c.bg_transparent>&lrm;아유, 고맙습니다, 예</c.bg_transparent></c.korean>

12
00:02:22,767 --> 00:02:24,561
<c.korean><c.bg_transparent>&lrm;아이고, 덥다, 저기, 저기…</c.bg_transparent></c.korean>
<c.korean><c.bg_transparent>&lrm;[안내 음성이 흘러나온다]</c.bg_transparent></c.korean>
```

## Merged srt Data looks like

```xml
12
00:02:19,889 --> 00:02:21,641
<c.korean><c.bg_transparent>&lrm;아유, 고맙습니다, 예</c.bg_transparent></c.korean>
&lrm;電梯門要開了
&lrm;謝謝妳

13
00:02:22,767 --> 00:02:24,561
<c.korean><c.bg_transparent>&lrm;아이고, 덥다, 저기, 저기…</c.bg_transparent></c.korean>
<c.korean><c.bg_transparent>&lrm;[안내 음성이 흘러나온다]</c.bg_transparent></c.korean>
&lrm;-唉唷，好熱喔，麻煩幫我按…
&lrm;-電梯門要關了
```